$$P(T>t+\tau | P>t) = \frac{f(t+\tau) \int_{t+\tau}^{\infty} f(x) dx}{f(t) \int_t^{\infty} f(x) dx}$$

In [44]:
import numpy as np
from scipy import linalg

In [45]:
P = np.array([[0.1, 0.2, 0.7],
              [0.4, 0.5, 0.1],
              [0.0, 0.1, 0.9]])


In [46]:
if not np.allclose(P.sum(axis=1), np.ones(P.shape[0])):
    print('Invalid probability transition matrix')


In [47]:
eigenvalues, eigenvectors = linalg.eig(P)
print(eigenvalues,"\n",eigenvectors)


[0. +0.j 0.5+0.j 1. +0.j] 
 [[ 0.77204865  0.06052275  0.57735027]
 [-0.63167617  0.96836405  0.57735027]
 [ 0.07018624 -0.24209101  0.57735027]]


In [48]:
eigenvector = eigenvectors[:, np.isclose(eigenvalues, 1)]
eigenvector = eigenvector[:, np.all(eigenvector >= 0)]
print(eigenvector)

[[[0.57735027]]

 [[0.57735027]]

 [[0.57735027]]]


In [49]:
if eigenvector.size == 0:
    eigenvector = np.ones(P.shape[0])
    for _ in range(100):
        eigenvector = np.dot(eigenvector, P)

print(eigenvector)


[[[0.57735027]]

 [[0.57735027]]

 [[0.57735027]]]


In [50]:
stationary_distribution = eigenvector / eigenvector.sum()
eigenvector.sum()

1.7320508075688776

In [51]:
print('Stationary distribution:\n', stationary_distribution)


Stationary distribution:
 [[[0.33333333]]

 [[0.33333333]]

 [[0.33333333]]]


In [61]:
# Define the matrix A
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

# Compute the QR decomposition of A
Q, R = np.linalg.qr(A)

# Solve the overdetermined system Ax = b
b = np.array([1, 2, 3, 4])
x = np.linalg.solve(R, Q.T @ b)

print(x)  # prints the solution to the system


[-0.06948425  0.13896851  0.26384908]


In [60]:
# Define the matrix A
A = np.array([[0.7, 0.2, 0.1],
              [0.4, 0.6, 0.0],
              [0.0, 1.0, 0.0],
              [1.0, 1.0, 1.0]])
b = np.array([0., 0., 0., 1.])

x_lstsq = np.linalg.lstsq(A,b)[0] # computing the numpy solution

Q,R = np.linalg.qr(A) # qr decomposition of A
Qb = np.dot(Q.T, b) # computing Q^T*b (project b onto the range of A)
x_qr = np.linalg.solve(R,Qb) # solving R*x = Q^T*b

# comparing the solutions
print('qr solution')
print(x_qr)
print('lstqs solution')
print(x_lstsq)

qr solution
[-0.12692308  0.02051282  1.10384615]
lstqs solution
[-0.12692308  0.02051282  1.10384615]


/tmp/ipykernel_60978/3948617761.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x_lstsq = np.linalg.lstsq(A,b)[0] # computing the numpy solution


In [64]:
# Define the transition matrix
P = np.array([[0.1, 0.2, 0.7],
              [0.4, 0.5, 0.1],
              [0.0, 0.1, 0.9]])

# Define the initial state vector
x0 = np.array([0.0, 0.9, 0.1])

# Check that the transition matrix is a valid probability matrix
if not np.allclose(P.sum(axis=1), np.ones(P.shape[0])):
    raise ValueError("Transition matrix is not a valid probability matrix")

# Check that the initial state vector is a valid probability vector
if not np.allclose(x0.sum(), 1) or np.any(x0 < 0):
    raise ValueError("Initial state vector is not a valid probability vector")

# Set a tolerance for the convergence criterion
tol = 1e-8

# Set the initial state vector as the current state vector
x = x0

# Iterate until the state vectors converge to a fixed point
while True:
    # Compute the next state vector
    x_next = P @ x

    # Check if the next state vector is close enough to the current state vector
    if np.allclose(x_next, x, rtol=tol, atol=tol):
        break

    # Update the current state vector
    x = x_next

# Print the stationary distribution
print(x)


[0.236      0.23600001 0.236     ]


In [66]:
def stationary(transition):
    # Check that the transition matrix is square, has non-negative entries,
    # and that all entries are between 0 and 1
    if not (isinstance(transition, np.ndarray) and
            transition.shape[0] == transition.shape[1] and
            np.all(transition >= 0) and np.all(transition <= 1)):
        raise ValueError("Invalid transition matrix")

    # Compute the matrix p
    p = np.diag(transition.shape[0]) - transition

    # Concatenate the matrices A and b
    A = np.vstack((p.T, np.ones(transition.shape[1])))
    b = np.hstack((np.zeros(transition.shape[0]), 1))

    # Solve the linear system Ax = b
    res = np.linalg.solve(A, b)

    # Set the names of the entries in res
    res_names = ["state.{}".format(i+1) for i in range(transition.shape[0])]

    return dict(zip(res_names, res))

# Test the function with the given matrix
transition = np.array([[0.7, 0.2, 0.1], [0.4, 0.6, 0], [0, 0, 1]])
print(stationary(transition))


ValueError: Input must be 1- or 2-d.